In [207]:
import pandas as pd
import os
import ast
import shutil

In [208]:
def erase_files(path):

    for filename in os.listdir(path):

        file_path = os.path.join(path, filename)

        try:
            if os.path.isfile(file_path): os.remove(file_path)
        except Exception as e:
            print(f"Error deleting {file_path}: {e}")

**Grouping csv files in one annotations.csv for each dataset**

In [209]:
path = "/home/martin/Desktop/visdecode+"
folders = os.listdir(path)

for folder in folders:

    csv_folder_path = os.path.join(path, folder + "/csv")
    if os.path.exists(csv_folder_path):

        csv_files = os.listdir(csv_folder_path)
        df = None

        for file in csv_files:

            df2 = pd.read_csv(os.path.join(csv_folder_path, file))

            if type(df) == type(None): 
                df = df2
            else:
                df = pd.concat([df, df2], ignore_index=True)

        df.to_csv(os.path.join(path, folder + "/annotations.csv"))

**Grouping all annotations.csv in one file**

In [210]:
folders = os.listdir(path)
df = None

for folder in folders:

    folder_path = os.path.join(path, folder)
    if folder != "output" and folder != "png" and os.path.isdir(folder_path):

        df2 = pd.read_csv(folder_path + "/annotations.csv", index_col = 0)

        if type(df) == type(None):
            df = df2
        else:
            df = pd.concat([df, df2], ignore_index=True)

df["x_name"] = df["x_name"].astype(object).fillna(" ")
df["y_name"] = df["y_name"].astype(object).fillna(" ")

colors = [colors_str[1:-1].replace(",","").replace("'","") for colors_str in df["colors"]]
df["colors"] = colors

df.to_csv(path + "/all_annotations.csv")
df["mark_type"].value_counts()

mark_type
vbar     6916
hbar     6720
pie      4078
line     1693
point    1139
Name: count, dtype: int64

**Sampling super dataset**

In [211]:
df = pd.read_csv(path + "/all_annotations.csv", index_col = 0)
sampled_df = df.sample(1000, random_state = 42)

erase_files(f"{path}/png")

for idx, row in sampled_df.iterrows():
        
    original_image = row["original_image"]
    dataset = row["dataset_name"]

    src_path = os.path.join(f"{path}/{dataset}/png", original_image)
    dest_path = os.path.join(f"{path}/png", original_image)

    try:
        shutil.copy(src_path, dest_path)
    except IOError as e:
        print(f"Failed to copy {original_image}: {e}")

sampled_df.to_csv(path + "/annotations.csv")
sampled_df["mark_type"].value_counts()

mark_type
vbar     325
hbar     322
pie      211
line      76
point     66
Name: count, dtype: int64